# Marvel Social Network Analysis

In [57]:
! ls

HW2 - Q1.Rmd                  Q2H - Shan Holly analysis.Rmd
HW2 - Q2.Rmd                  README.md
Homework_3_template.Rmd       data
Marvel_Network.ipynb          hw4_solns.Rmd
Part_G.Rmd                    images
Part_g.R


In [58]:
import numpy as np
import pandas as pd

In [59]:
heros = pd.read_csv("data/edges.csv")  # hero, comic
hero_edges = pd.read_csv("data/hero-network.csv")  # hero 1, hero 2

In [60]:
heros.head(3)

,hero,comic
0,24-HOUR MAN/EMMANUEL,AA2 35
1,3-D MAN/CHARLES CHAN,AVF 4
2,3-D MAN/CHARLES CHAN,AVF 5


In [61]:
hero_edges.head(3)

,hero1,hero2
0,"LITTLE, ABNER",PRINCESS ZANDA
1,"LITTLE, ABNER",BLACK PANTHER/T'CHAL
2,BLACK PANTHER/T'CHAL,PRINCESS ZANDA


### Making a weighted graph  
We will process the heros data set to give weights to the nodes, for the number of comics that each character appears in.   
  
We can also use this same data set to see how often characters cross paths, providing edge weights

### Deduplicate edges to generate weights  
Below, I loop through each row of the "hero_edges" data frame, where each row represents a a co-appearance of two comic book characters. With this, we can count the number of times that each character appears (node weights) and the number of times that each hero pair appears (edge weights).

In [62]:
from collections import defaultdict

In [63]:
edge_weights = defaultdict(int)
node_weights = defaultdict(int)

i=0
for row in hero_edges.itertuples():
    
    # grab the two heros names and remove commas, white space
    h1, h2 = row[1].replace(",", "").strip(), row[2].replace(",", "").strip() 
    
    # handle edges that contain self references
    if h1 != h2:
        # update node weights for individual hero appearance counts
        node_weights[h1] += 1
        node_weights[h2] += 1


        # update edge weights for hero co-appearence
        hero_list = [h1, h2]  # put the heros in a list
        hero_list.sort()  # sort the list for naming consistency
        hero_tup = (hero_list[0], hero_list[1])  # use as a key
        edge_weights[hero_tup] += 1  # increment co-appearence count

### Weighted edges  to csv
Hero 1, hero 2, number of co-occurrences

In [64]:
len(edge_weights.keys())

167100

In [70]:
edges_tup = zip(edge_weights.keys(), edge_weights.values())
edges = [[k[0], k[1], v] for k, v in edges_tup]

In [71]:
weighted_edge = pd.DataFrame(edges, columns = ["hero1", "hero2", "weight"])
weighted_edge.to_csv("data/weighted_edges.csv", index=False)

In [72]:
weighted_edge.head()

,hero1,hero2,weight
0,HATCHWAY VERNON,METEORITE III/VALERI,1
1,INVISIBLE WOMAN/SUE,PULSSUS,4
2,DIAMOND KATHY,FURY COL. NICHOLAS,1
3,PIPELINE/CORMICK GRI,SHADOW KING/AMAHL FA,1
4,NIGHTCRAWLER/KURT WA,WONG,1


### Weighted nodes  to csv
Hero, number of occurrences

In [73]:
len(node_weights.keys())

6421

In [74]:
nodes = zip(node_weights.keys(), node_weights.values())

In [75]:
weighted_nodes = pd.DataFrame(nodes, columns = ["hero", "weight"])
weighted_nodes.to_csv("data/weighted_nodes.csv", index=False)

In [82]:
weighted_nodes.head()

,hero,weight
0,JARVIS EDWIN | TIME,25
1,KITAGOWA DR. KITTY,8
2,ZEITGEIST II/AXEL CL,10
3,AUSTIN TERRY,40
4,PIECEMEAL/GILBERT BE,102


### Create data frames of the weighted nodes and edges

In [88]:
node_df = pd.read_csv("data/weighted_nodes.csv")
node_df.sort_values("weight", ascending = 0).head()

,hero,weight
4394,CAPTAIN AMERICA,16259
4717,SPIDER-MAN/PETER PAR,13717
6394,IRON MAN/TONY STARK,11817
4407,THOR/DR. DONALD BLAK,11427
2571,THING/BENJAMIN J. GR,10681


In [87]:
edge_df = pd.read_csv("data/weighted_edges.csv")
edge_df.sort_values("weight", ascending = 0).head()

,hero1,hero2,weight
57190,MISS AMERICA/MADELIN,PATRIOT/JEFF MACE,1894
134528,HUMAN TORCH/JOHNNY S,THING/BENJAMIN J. GR,744
8026,HUMAN TORCH/JOHNNY S,MR. FANTASTIC/REED R,713
141326,MR. FANTASTIC/REED R,THING/BENJAMIN J. GR,708
59245,INVISIBLE WOMAN/SUE,MR. FANTASTIC/REED R,701


### Shrink the network to top 1000 common characters

In [89]:
top_1000_characters = node_df.sort_values("weight", ascending = 0).head(1000)

In [154]:
top_1000_characters.to_csv(path_or_buf="data/weighted_nodes_top_1000.csv", sep=',', index = False)

In [155]:
popular_edges = edge_df.loc[edge_df.hero1.isin(top_1000_characters.hero) & \
                            edge_df.hero2.isin(top_1000_characters.hero)]

In [156]:
popular_edges.shape

(67461, 3)

In [157]:
popular_edges.head()

,hero1,hero2,weight
3,PIPELINE/CORMICK GRI,SHADOW KING/AMAHL FA,1
4,NIGHTCRAWLER/KURT WA,WONG,1
8,NIGHTCRAWLER/KURT WA,PIERCE DONALD,4
9,SABRETOOTH/VICTOR CR,SELENE,1
10,HUDSON HEATHER,THOR III DOPPELGANGE,2


In [158]:
popular_edges.to_csv(path_or_buf="data/weighted_edge_top_1000.csv", sep=',', index = False)

### Put this all into a network x graph object and visualize with plotly

In [227]:
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import plotly.plotly as py
from plotly.graph_objs import *
from plotly.offline import init_notebook_mode, iplot

init_notebook_mode(True)
% matplotlib inline

In [160]:
marvel_edges = pd.read_csv("data/weighted_edge_top_1000.csv", header = 0)
marvel_nodes = pd.read_csv("data/weighted_nodes_top_1000.csv", header = 0)

In [161]:
marvel_edges.shape

(67461, 3)

In [162]:
marvel_edges.head()

,hero1,hero2,weight
0,PIPELINE/CORMICK GRI,SHADOW KING/AMAHL FA,1
1,NIGHTCRAWLER/KURT WA,WONG,1
2,NIGHTCRAWLER/KURT WA,PIERCE DONALD,4
3,SABRETOOTH/VICTOR CR,SELENE,1
4,HUDSON HEATHER,THOR III DOPPELGANGE,2


### Top 100 characters by degree

In [163]:
top_100_degree = marvel_nodes.sort_values("weight", ascending = 0).head(100)

In [164]:
top_100_degree.head()

,hero,weight
0,CAPTAIN AMERICA,16259
1,SPIDER-MAN/PETER PAR,13717
2,IRON MAN/TONY STARK,11817
3,THOR/DR. DONALD BLAK,11427
4,THING/BENJAMIN J. GR,10681


In [165]:
popular_edges_degree = marvel_edges.loc[marvel_edges.hero1.isin(top_100_degree.hero) & \
                                 marvel_edges.hero2.isin(top_100_degree.hero)]

In [201]:
G2 = nx.DiGraph()
for idx, row in popular_edges_degree.iterrows():
    # log scale down the weight. its way too high
    G2.add_edge(row['hero1'], row['hero2'], weight=np.log(int(row['weight'])))

In [175]:
pos2 = nx.spring_layout(G2)  # positions for all nodes

In [176]:
for p in pos2:
    x, y = pos2[p]
    G2.node[p]['pos'] = (x,y)

In [177]:
edge_trace2 = Scatter(
    x=[],
    y=[],
    line=Line(width=0.5,color='#888'),
    hoverinfo='none',
    mode='lines')

for edge in G2.edges():
    x0, y0 = G2.node[edge[0]]['pos']
    x1, y1 = G2.node[edge[1]]['pos']
    edge_trace2['x'] += [x0, x1, None]
    edge_trace2['y'] += [y0, y1, None]

In [178]:
node_trace2 = Scatter(
    x=[],
    y=[],
    text=[],
    mode='markers',
    hoverinfo='text',
    marker=Marker(
        showscale=True,
        colorscale='Electric',
        reversescale=True,
        color=[],
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line=dict(width=2)))

for node in G2.nodes():
    x, y = G2.node[node]['pos']
    node_trace2['x'].append(x)
    node_trace2['y'].append(y)
    # node_trace2['text'].append(node)

In [179]:
for node, adjacencies in enumerate(G2.adjacency()):
    num_conn = len(adjacencies[1].keys())  # how many connections. Python 2 version
    
    node_trace2['marker']['color'].append(num_conn)
    node_info = '# of connections: '+str(num_conn)
    
    name_info = adjacencies[0]
    
    node_trace2['text'].append(name_info + "<br>" +node_info)

In [282]:
fig2 = Figure(data=Data([edge_trace2, node_trace2]),
             layout=Layout(
                title='<br>Marvel Social Network Top 100 Most Popular Characters',
                titlefont=dict(size=16),
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    text="Python code: <a href='https://plot.ly/ipython-notebooks/network-graphs/'> https://plot.ly/ipython-notebooks/network-graphs/</a>",
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002 ) ],
                xaxis=XAxis(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=YAxis(showgrid=False, zeroline=False, showticklabels=False)))

py.plot(fig2, "marvel_graph_degree.html")

u'https://plot.ly/~tjpell/143'

In [283]:
py.iplot(fig2)

### Lets do the same graphic, for betweenness

In [256]:
between_100 = pd.read_csv("data/between_top_100.csv", names=["hero", "betweenness"]).tail(100)
between_100.head()

,hero,betweenness
1,SPIDER-MAN/PETER PAR,2676603.25169885
2,CAPTAIN AMERICA,1935707.52353221
3,IRON MAN/TONY STARK,1067304.54569492
4,WOLVERINE/LOGAN,1062680.33196175
5,DR. STRANGE/STEPHEN,846621.342793805


In [257]:
between_100.shape

(100, 2)

In [258]:
popular_edges_between = marvel_edges.loc[marvel_edges.hero1.isin(between_100.hero) & \
                                 marvel_edges.hero2.isin(between_100.hero)]

In [259]:
popular_edges_between.head()

,hero1,hero2,weight
33,WOLFSBANE/RAHNE SINC,X-MAN/NATHAN GREY,2
40,CAPTAIN AMERICA,MOON KNIGHT/MARC SPE,13
46,STARSHINE II/BRANDY,SUNSPOT/ROBERTO DACO,1
76,DAREDEVIL/MATT MURDO,FIRESTAR/ANGELICA JO,13
142,HAVOK/ALEX SUMMERS,ROGUE /,78


In [260]:
popular_edges_between.shape

(3040, 3)

In [289]:
G3 = nx.DiGraph()
for idx, row in popular_edges_between.iterrows():
    # log scale down the weight. its way too high
    G3.add_edge(row['hero1'], row['hero2'])

In [290]:
pos3 = nx.spring_layout(G3)  # positions for all nodes

In [291]:
for p in pos3:
    x, y = pos3[p]
    G3.node[p]['pos'] = (x,y)
    G3.node[p]['between'] = between_100.loc[between_100.hero == p].betweenness

In [292]:
edge_trace3 = Scatter(
    x=[],
    y=[],
    line=Line(width=0.5,color='#888'),
    hoverinfo='none',
    mode='lines')

for edge in G3.edges():
    x0, y0 = G3.node[edge[0]]['pos']
    x1, y1 = G3.node[edge[1]]['pos']
    edge_trace3['x'] += [x0, x1, None]
    edge_trace3['y'] += [y0, y1, None]

In [293]:
node_trace3 = Scatter(
    x=[],
    y=[],
    text=[],
    mode='markers',
    hoverinfo='text',
    marker=Marker(
        showscale=True,
        colorscale='Bluered',
        reversescale=True,
        color=[],
        size=10,
        colorbar=dict(
            thickness=15,
            title='Degree of Betweenness',
            xanchor='left',
            titleside='right'
        ),
        line=dict(width=2)))

for node in G3.nodes():
    x, y = G3.node[node]['pos']
    between = G3.node[node]['between']
    node_trace3['x'].append(x)
    node_trace3['y'].append(y)

In [294]:
for node, adjacencies in enumerate(G3.adjacency()):
    num_conn = len(adjacencies[1].keys())  # how many connections. Python 2 version
    between = G3.node[adjacencies[0]]['between']
    bet = str(between).split("    ")[1].split("\n")[0]
        
    node_trace3['marker']['color'].append(float(bet))
    node_info = 'Degree of betweenness: '+str(float(bet))
    
    name_info = adjacencies[0]
    
    node_trace3['text'].append(name_info + "<br>" +node_info)

In [295]:
fig3 = Figure(data=Data([edge_trace3, node_trace3]),
             layout=Layout(
                title='<br>Marvel Social Network Top 100 Most Popular Characters, Betweenness',
                titlefont=dict(size=16),
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    text="Python code: <a href='https://plot.ly/ipython-notebooks/network-graphs/'> https://plot.ly/ipython-notebooks/network-graphs/</a>",
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002 ) ],
                xaxis=XAxis(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=YAxis(showgrid=False, zeroline=False, showticklabels=False)))

py.plot(fig3, "marvel_graph_between.html")

u'https://plot.ly/~tjpell/155'

In [296]:
py.iplot(fig3)

### We can now repeat the same process, for eigen centrality

In [234]:
eigs_100 = pd.read_csv("data/eigen_top_100.csv", names=["hero", "betweenness"]).tail(100)
eigs_100.head()

,hero,betweenness
1,CAPTAIN AMERICA,1
2,THING/BENJAMIN J. GR,0.826634868166242
3,HUMAN TORCH/JOHNNY S,0.812159350669197
4,MR. FANTASTIC/REED R,0.798075131298306
5,IRON MAN/TONY STARK,0.795150288522563


In [236]:
popular_edges_eigs = marvel_edges.loc[marvel_edges.hero1.isin(eigs_100.hero) & \
                                 marvel_edges.hero2.isin(eigs_100.hero)]

In [237]:
popular_edges_eigs.head()

,hero1,hero2,weight
47,BLACK WIDOW/NATASHA,KARNAK [INHUMAN],5
76,DAREDEVIL/MATT MURDO,FIRESTAR/ANGELICA JO,13
82,HAWK,LOCKJAW [INHUMAN],10
92,BINARY/CAROL DANVERS,JUSTICE II/VANCE AST,10
110,GALACTUS/GALAN,SILVER SURFER/NORRIN,70


In [316]:
G4 = nx.DiGraph()
for idx, row in popular_edges_eigs.iterrows():
    # log scale down the weight. its way too high
    G4.add_edge(row['hero1'], row['hero2'])

In [317]:
pos4 = nx.spring_layout(G4)  # positions for all nodes

In [318]:
for p in pos4:
    x, y = pos4[p]
    G4.node[p]['pos'] = (x,y)
    G4.node[p]['eigs'] = eigs_100.loc[eigs_100.hero == p].betweenness

In [319]:
edge_trace4 = Scatter(
    x=[],
    y=[],
    line=Line(width=0.5,color='#888'),
    hoverinfo='none',
    mode='lines')

for edge in G4.edges():
    x0, y0 = G4.node[edge[0]]['pos']
    x1, y1 = G4.node[edge[1]]['pos']
    edge_trace4['x'] += [x0, x1, None]
    edge_trace4['y'] += [y0, y1, None]

In [320]:
node_trace4 = Scatter(
    x=[],
    y=[],
    text=[],
    mode='markers',
    hoverinfo='text',
    marker=Marker(
        showscale=True,
        colorscale='Earth',
        reversescale=True,
        color=[],
        size=10,
        colorbar=dict(
            thickness=15,
            title='Degree of Betweenness',
            xanchor='left',
            titleside='right'
        ),
        line=dict(width=2)))

for node in G4.nodes():
    x, y = G4.node[node]['pos']
    between = G4.node[node]['eigs']
    node_trace4['x'].append(x)
    node_trace4['y'].append(y)

In [321]:
for node, adjacencies in enumerate(G4.adjacency()):
    num_conn = len(adjacencies[1].keys())  # how many connections. Python 2 version
    eig = G4.node[adjacencies[0]]['eigs']
    
    e = str(eig).split("    ")[1].split("\n")[0]
    
    node_trace4['marker']['color'].append(e)
    node_info = 'Degree of Eigen-centrality: '+str(e) # + "<br># of connections: " + str(num_conn)
    
    name_info = adjacencies[0]
    
    node_trace4['text'].append(name_info + "<br>" +node_info)

In [322]:
fig4 = Figure(data=Data([edge_trace4, node_trace4]),
             layout=Layout(
                title='<br>Marvel Social Network Top 100 Most Popular Characters, Eigen-Centrality',
                titlefont=dict(size=16),
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    text="Python code: <a href='https://plot.ly/ipython-notebooks/network-graphs/'> https://plot.ly/ipython-notebooks/network-graphs/</a>",
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002 ) ],
                xaxis=XAxis(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=YAxis(showgrid=False, zeroline=False, showticklabels=False)))

py.plot(fig4, "marvel_graph_eig.html")

u'https://plot.ly/~tjpell/171'

In [315]:
py.iplot(fig4)